In [1]:
import pandas as pd
import numpy as np


In [2]:
#Retrieve dataframe
df = pd.read_csv('../../Data/Mobile/train.csv')

In [3]:
#Look at size and feature dtypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40428967 entries, 0 to 40428966
Data columns (total 24 columns):
id                  float64
click               int64
hour                int64
C1                  int64
banner_pos          int64
site_id             object
site_domain         object
site_category       object
app_id              object
app_domain          object
app_category        object
device_id           object
device_ip           object
device_model        object
device_type         int64
device_conn_type    int64
C14                 int64
C15                 int64
C16                 int64
C17                 int64
C18                 int64
C19                 int64
C20                 int64
C21                 int64
dtypes: float64(1), int64(14), object(9)
memory usage: 7.2+ GB


In [5]:
#Algorithm switches datatypes to smaller dtypes that will shrink the memory usage

def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist

In [6]:
# Get shrunken data and delete original df
dfred = reduce_mem_usage(df)
del df

Memory usage of properties dataframe is : 7402.765029907227  MB
******************************
Column:  id
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  click
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  hour
dtype before:  int64
dtype after:  uint32
******************************
******************************
Column:  C1
dtype before:  int64
dtype after:  uint16
******************************
******************************
Column:  banner_pos
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  device_type
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  device_conn_type
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  C14
dtype before:  int64
dtype after:  uint16
***

In [7]:
#Set the shrunken data to the a pandas df and delete old shrunken data
df = dfred[0]

del dfred

In [8]:
# Save as pickle file to ensure the dataframe keeps the data saving datatypes
df.to_pickle('../../Data/Mobile/train1.pkl')